In [ ]:
import netCDF4
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
from matplotlib import pyplot as plt
import scipy.stats as sp
import statistics as stats

In [ ]:
# R**2 Data Storage

r_squared_storage = ['None'] * 2
a_value_storage = ['None'] * 2
b_value_storage = ['None'] * 2
x_storage = ['None'] * 2
y_storage = ['None'] * 2

In [ ]:
## SSP NAWH Index

## Importing SSTA
for f in range(0,2):
    filesetssp = '[WH INDEX SSP PATH]'
    sspAMOCData = '[SSP AMOC INDEX PATH]'

    sspsst = (xr.open_dataset(filesetssp))


    ## Importing AMOC Index

    ssp_amoc = (xr.open_dataset(sspAMOCData))

    ssp_amoc = ssp_amoc.msftmz
    ssp_amoc = ssp_amoc/1e9


    ##ANOMALY CALC
    sspsst_anomaly = sspsst - sspsst.mean(dim="year")
    amoc_anomaly = ssp_amoc - ssp_amoc.mean(dim="year")

    #ssp fix
    sspsst_anomaly = sspsst_anomaly.tos[:,:85]

    sspsst_anomaly = sspsst_anomaly[f]
    amoc_anomaly = amoc_anomaly[f]

    #Year Change

    a, b, r, p, err = sp.stats.linregress(amoc_anomaly, sspsst_anomaly)
    print("The M value is:", a)
    print("The B value is:", b)
    print("The R value is:", r**2)
    print("The P value is:", p)

    r_squared_storage[f] = r**2
    a_value_storage[f] = a
    b_value_storage[f] = b

    ## Linear Regression Calculation

    def estimate_coef(x, y):
        # number of observations/points
        n = np.size(x)
    
    
        # mean of x and y vector
        m_x = np.mean(x)
        m_y = np.mean(y)
    
        # calculating cross-deviation and deviation about x
        SS_xy = np.sum(y*x) - n*m_y*m_x
        SS_xx = np.sum(x*x) - n*m_x*m_x
    
        # calculating regression coefficients
        M = SS_xy / SS_xx
        B = m_y - M*m_x
    
        return (B, M)

    def plot_regression_line(x, y, b):
        # plotting the actual points as scatter plot
        plt.scatter(x, y, color = "k",
                marker = "s", s = 9, alpha = 0.5)
    
        # predicted response vector
        y_pred = b[0] + b[1]*x
    
        # plotting the regression line
        plt.plot(x, y_pred, color = "r")
    
        # putting labels

        plt.xlim(-4, 8)
        plt.ylim(-2.5, 2.5)
        plt.xlabel('Sverdrup (Sv)', fontsize = 13)
        plt.ylabel('WH Index Anomaly', fontsize = 13)
        plt.rcParams.update({'font.family':'sans-serif'})
        plt.rcParams.update({'font.sans-serif':'Arial'})
        plt.rcParams.update({'font.size': 13})
        plt.minorticks_on()
    
        # function to show plot
        r_graphed_txt = "R Squared Value: "
        r_graphed_data = '%.3f'%(r**2)
        r_graphed = r_graphed_txt + r_graphed_data
        plt.title("SSP WH Index vs Sverdrup - Rlzn {}".format(f+1))
        plt.text(3, -2.3, s=r_graphed)
        plt.show()


    def main():
        # observations / data
        x = np.array(amoc_anomaly)
        y = np.array(sspsst_anomaly)

        x_storage[f] = x
        y_storage[f] = y
    
        # estimating coefficients
        lin = estimate_coef(x, y)
        
    
        # plotting regression line
        plot_regression_line(x, y, lin)
        
    if __name__ == "__main__":
        main()

##-------------------------------------------------------
        
r_squared_mean = np.average(r_squared_storage, axis=0)
stdev_r2 = stats.stdev(r_squared_storage)

a_mean = np.average(a_value_storage, axis=0)
b_mean = np.average(b_value_storage, axis=0)

x_append = x_storage[0]
y_append = y_storage[0]

for j in range(1,2):
    x_append = np.append(x_append, x_storage[j])
    y_append = np.append(y_append, y_storage[j])

##-------------------------------------------------------

# Graph creation
plt.scatter(x_append, y_append, s = 9, marker = 's', color = "k", alpha=0.25)
plt.axline((0,b_mean), slope=a_mean, linewidth = 2, color='fuchsia')

#max and min
plt.axline((0,b_value_storage[0]), slope=a_value_storage[0], linewidth = 1, color='orange', alpha = 0.4) #max
plt.axline((0,b_value_storage[1]), slope=a_value_storage[1], linewidth = 1, color='mediumpurple', alpha = 0.4) #min

plt.xlim(-4, 4)
plt.ylim(-2.5, 2.5)
plt.xlabel('Sverdrup (Sv)', fontsize = 13)
plt.ylabel('WH Index Anomaly', fontsize = 13)
plt.rcParams.update({'font.family':'sans-serif'})
plt.rcParams.update({'font.sans-serif':'Arial'})
plt.rcParams.update({'font.size': 13})
plt.minorticks_on()

# function to show plot
r_plot = "R^2 = ", '%.3f'%(r_squared_mean)
r_max = "R^2 Max = ", '%.3f'%(max(r_squared_storage))
r_min = "R^2 Min = ", '%.3f'%(min(r_squared_storage))
err_plot = "R^2 StDev = ", '%.3f'%(stdev_r2)
plt.text(1.0, -2.3, s=str(r_plot), color = 'fuchsia')
plt.text(1.0, -1.9, s=str(r_max), color = 'orange')
plt.text(1.0, -2.1, s=str(r_min), color = 'mediumpurple')
plt.text(-3.9, -2.3, s=str(err_plot))
plt.title('NAWH Index Correlation - Both Realizations', y=1)
fig_dir = '[EXPORT PATH]'
fig_namesave = 'SSP NAWH Index Regression'
plt.savefig(fig_dir + fig_namesave, dpi = 600, facecolor='white', transparent=False)
plt.show()

In [ ]:
## SSP Global SST

## Importing SSTA
for f in range(0,2):
    filesetssp = '[non-regridded TOS SSP PATH]'
    sspAMOCData = '[SSP AMOC INDEX PATH]'

    sspsst = (xr.open_dataset(filesetssp))

    GLOBAL = sspsst.where(sspsst.latitude < 75).where(sspsst.longitude < 360).where(sspsst.longitude > 0).where(sspsst.latitude > -60).mean(("i", "j"))
 
    sspsst = GLOBAL

    ## Importing AMOC Index

    ssp_amoc = (xr.open_dataset(sspAMOCData))

    ssp_amoc = ssp_amoc.msftmz
    ssp_amoc = ssp_amoc/1e9


    ##ANOMALY CALC
    sspsst_anomaly = sspsst - sspsst.mean(dim="year")
    amoc_anomaly = ssp_amoc - ssp_amoc.mean(dim="year")

    #ssp fix
    sspsst_anomaly = sspsst_anomaly.tos[:,:85]

    sspsst_anomaly = sspsst_anomaly[f]
    amoc_anomaly = amoc_anomaly[f]

    #Year Change

    a, b, r, p, err = sp.stats.linregress(amoc_anomaly, sspsst_anomaly)
    print("The M value is:", a)
    print("The B value is:", b)
    print("The R value is:", r**2)
    print("The P value is:", p)

    r_squared_storage[f] = r**2
    a_value_storage[f] = a
    b_value_storage[f] = b

    ## Linear Regression Calculation

    def estimate_coef(x, y):
        # number of observations/points
        n = np.size(x)
    
    
        # mean of x and y vector
        m_x = np.mean(x)
        m_y = np.mean(y)
    
        # calculating cross-deviation and deviation about x
        SS_xy = np.sum(y*x) - n*m_y*m_x
        SS_xx = np.sum(x*x) - n*m_x*m_x
    
        # calculating regression coefficients
        M = SS_xy / SS_xx
        B = m_y - M*m_x
    
        return (B, M)

    def plot_regression_line(x, y, b):
        # plotting the actual points as scatter plot
        plt.scatter(x, y, color = "k",
                marker = "s", s = 9, alpha = 0.5)
    
        # predicted response vector
        y_pred = b[0] + b[1]*x
    
        # plotting the regression line
        plt.plot(x, y_pred, color = "r")
    
        # putting labels

        plt.xlim(-4, 8)
        plt.ylim(-2.5, 2.5)
        plt.xlabel('Sverdrup (Sv)', fontsize = 13)
        plt.ylabel('Global SST Anomaly', fontsize = 13)
        plt.rcParams.update({'font.family':'sans-serif'})
        plt.rcParams.update({'font.sans-serif':'Arial'})
        plt.rcParams.update({'font.size': 13})
        plt.minorticks_on()
    
        # function to show plot
        r_graphed_txt = "R Squared Value: "
        r_graphed_data = '%.3f'%(r**2)
        r_graphed = r_graphed_txt + r_graphed_data
        plt.title("SSP Global SST vs Sverdrup - Rlzn {}".format(f+1))
        plt.text(3, -2.3, s=r_graphed)
        plt.show()


    def main():
        # observations / data
        x = np.array(amoc_anomaly)
        y = np.array(sspsst_anomaly)

        x_storage[f] = x
        y_storage[f] = y
    
        # estimating coefficients
        lin = estimate_coef(x, y)
        
    
        # plotting regression line
        plot_regression_line(x, y, lin)
        
    if __name__ == "__main__":
        main()

##-------------------------------------------------------
        
r_squared_mean = np.average(r_squared_storage, axis=0)
stdev_r2 = stats.stdev(r_squared_storage)

a_mean = np.average(a_value_storage, axis=0)
b_mean = np.average(b_value_storage, axis=0)

x_append = x_storage[0]
y_append = y_storage[0]

for j in range(1,2):
    x_append = np.append(x_append, x_storage[j])
    y_append = np.append(y_append, y_storage[j])

##-------------------------------------------------------

# Graph creation
plt.scatter(x_append, y_append, s = 9, marker = 's', color = "k", alpha=0.25)
plt.axline((0,b_mean), slope=a_mean, linewidth = 2, color='fuchsia')

#max and min
plt.axline((0,b_value_storage[0]), slope=a_value_storage[0], linewidth = 1, color='orange', alpha = 0.4) #max
plt.axline((0,b_value_storage[1]), slope=a_value_storage[1], linewidth = 1, color='mediumpurple', alpha = 0.4) #min

plt.xlim(-4, 4)
plt.ylim(-2.5, 2.5)
plt.xlabel('Sverdrup (Sv)', fontsize = 13)
plt.ylabel('Global SST Anomaly', fontsize = 13)
plt.rcParams.update({'font.family':'sans-serif'})
plt.rcParams.update({'font.sans-serif':'Arial'})
plt.rcParams.update({'font.size': 13})
plt.minorticks_on()

# function to show plot
r_plot = "R^2 = ", '%.3f'%(r_squared_mean)
r_max = "R^2 Max = ", '%.3f'%(max(r_squared_storage))
r_min = "R^2 Min = ", '%.3f'%(min(r_squared_storage))
err_plot = "R^2 StDev = ", '%.3f'%(stdev_r2)
plt.text(1.0, -2.3, s=str(r_plot), color = 'fuchsia')
plt.text(1.0, -1.9, s=str(r_max), color = 'orange')
plt.text(1.0, -2.1, s=str(r_min), color = 'mediumpurple')
plt.text(-3.9, -2.3, s=str(err_plot))
plt.title('Global SST Correlation - Both Realizations', y=1)
fig_dir = '[OUTPUT PATH]'
fig_namesave = 'SSP Global SST Regression'
plt.savefig(fig_dir + fig_namesave, dpi = 600, facecolor='white', transparent=False)
plt.show()

In [ ]:
## SSP WH SST

## Importing SSTA
for f in range(0,2):
    filesetssp = '[non-regridded TOS SSP PATH]'
    sspAMOCData = '[SSP AMOC INDEX PATH]'

    sspsst = (xr.open_dataset(filesetssp))

    NAWH = sspsst.where(sspsst.latitude < 61).where(sspsst.longitude < 345).where(sspsst.longitude > 322).where(sspsst.latitude > 48.5).mean(("i", "j"))
 
    sspsst = NAWH

    ## Importing AMOC Index

    ssp_amoc = (xr.open_dataset(sspAMOCData))

    ssp_amoc = ssp_amoc.msftmz
    ssp_amoc = ssp_amoc/1e9


    ##ANOMALY CALC
    sspsst_anomaly = sspsst - sspsst.mean(dim="year")
    amoc_anomaly = ssp_amoc - ssp_amoc.mean(dim="year")

    #ssp fix
    sspsst_anomaly = sspsst_anomaly.tos[:,:85]

    sspsst_anomaly = sspsst_anomaly[f]
    amoc_anomaly = amoc_anomaly[f]

    #Year Change

    a, b, r, p, err = sp.stats.linregress(amoc_anomaly, sspsst_anomaly)
    print("The M value is:", a)
    print("The B value is:", b)
    print("The R value is:", r**2)
    print("The P value is:", p)

    r_squared_storage[f] = r**2
    a_value_storage[f] = a
    b_value_storage[f] = b

    ## Linear Regression Calculation

    def estimate_coef(x, y):
        # number of observations/points
        n = np.size(x)
    
    
        # mean of x and y vector
        m_x = np.mean(x)
        m_y = np.mean(y)
    
        # calculating cross-deviation and deviation about x
        SS_xy = np.sum(y*x) - n*m_y*m_x
        SS_xx = np.sum(x*x) - n*m_x*m_x
    
        # calculating regression coefficients
        M = SS_xy / SS_xx
        B = m_y - M*m_x
    
        return (B, M)

    def plot_regression_line(x, y, b):
        # plotting the actual points as scatter plot
        plt.scatter(x, y, color = "k",
                marker = "s", s = 9, alpha = 0.5)
    
        # predicted response vector
        y_pred = b[0] + b[1]*x
    
        # plotting the regression line
        plt.plot(x, y_pred, color = "r")
    
        # putting labels

        plt.xlim(-4, 8)
        plt.ylim(-2.5, 2.5)
        plt.xlabel('Sverdrup (Sv)', fontsize = 13)
        plt.ylabel('WH SST Anomaly', fontsize = 13)
        plt.rcParams.update({'font.family':'sans-serif'})
        plt.rcParams.update({'font.sans-serif':'Arial'})
        plt.rcParams.update({'font.size': 13})
        plt.minorticks_on()
    
        # function to show plot
        r_graphed_txt = "R Squared Value: "
        r_graphed_data = '%.3f'%(r**2)
        r_graphed = r_graphed_txt + r_graphed_data
        plt.title("SSP WH SST vs Sverdrup - Rlzn {}".format(f+1))
        plt.text(3, -2.3, s=r_graphed)
        plt.show()


    def main():
        # observations / data
        x = np.array(amoc_anomaly)
        y = np.array(sspsst_anomaly)

        x_storage[f] = x
        y_storage[f] = y
    
        # estimating coefficients
        lin = estimate_coef(x, y)
        
    
        # plotting regression line
        plot_regression_line(x, y, lin)
        
    if __name__ == "__main__":
        main()

##-------------------------------------------------------
        
r_squared_mean = np.average(r_squared_storage, axis=0)
stdev_r2 = stats.stdev(r_squared_storage)

a_mean = np.average(a_value_storage, axis=0)
b_mean = np.average(b_value_storage, axis=0)

x_append = x_storage[0]
y_append = y_storage[0]

for j in range(1,2):
    x_append = np.append(x_append, x_storage[j])
    y_append = np.append(y_append, y_storage[j])

##-------------------------------------------------------

# Graph creation
plt.scatter(x_append, y_append, s = 9, marker = 's', color = "k", alpha=0.25)
plt.axline((0,b_mean), slope=a_mean, linewidth = 2, color='fuchsia')

#max and min
plt.axline((0,b_value_storage[0]), slope=a_value_storage[0], linewidth = 1, color='orange', alpha = 0.4) #max
plt.axline((0,b_value_storage[1]), slope=a_value_storage[1], linewidth = 1, color='mediumpurple', alpha = 0.4) #min

plt.xlim(-4, 4)
plt.ylim(-2.5, 2.5)
plt.xlabel('Sverdrup (Sv)', fontsize = 13)
plt.ylabel('WH SST Anomaly', fontsize = 13)
plt.rcParams.update({'font.family':'sans-serif'})
plt.rcParams.update({'font.sans-serif':'Arial'})
plt.rcParams.update({'font.size': 13})
plt.minorticks_on()

# function to show plot
r_plot = "R^2 = ", '%.3f'%(r_squared_mean)
r_max = "R^2 Max = ", '%.3f'%(max(r_squared_storage))
r_min = "R^2 Min = ", '%.3f'%(min(r_squared_storage))
err_plot = "R^2 StDev = ", '%.3f'%(stdev_r2)
plt.text(1.0, -2.3, s=str(r_plot), color = 'fuchsia')
plt.text(1.0, -1.9, s=str(r_max), color = 'orange')
plt.text(1.0, -2.1, s=str(r_min), color = 'mediumpurple')
plt.text(-3.9, -2.3, s=str(err_plot))
plt.title('WH SST Correlation - Both Realizations', y=1)
fig_dir = '[EXPORT PATH]'
fig_namesave = 'SSP WH SST Regression'
plt.savefig(fig_dir + fig_namesave, dpi = 600, facecolor='white', transparent=False)
plt.show()